In [6]:
from sklearn.datasets import load_digits
from sklearn.model_selection import train_test_split, cross_val_score, KFold, ShuffleSplit, GridSearchCV
from sklearn.svm import SVC
from sklearn.ensemble import RandomForestClassifier
from sklearn.metrics import classification_report, confusion_matrix, roc_auc_score, accuracy_score

In [3]:
dataset = load_digits()
X, y = dataset.data, dataset.target

model = RandomForestClassifier(random_state=42)

In [4]:
cv_scores = cross_val_score(model, X, y, cv=5, scoring='accuracy')
print(f"Средняя правильность перекрестной проверки: {cv_scores.mean():.4f}")

Средняя правильность перекрестной проверки: 0.9394


In [5]:
kfold5 = KFold(n_splits=5)
kfold10 = KFold(n_splits=10)

cv_scores = cross_val_score(model, X, y, cv=kfold5, scoring='accuracy')
print(f"Средняя правильность перекрестной проверки (kfold=5): {cv_scores.mean():.4f}")

cv_scores = cross_val_score(model, X, y, cv=10, scoring='accuracy')
print(f"Средняя правильность перекрестной проверки (cv=10): {cv_scores.mean():.4f}")

cv_scores = cross_val_score(model, X, y, cv=kfold10, scoring='accuracy')
print(f"Средняя правильность перекрестной проверки (kfold=10): {cv_scores.mean():.4f}")

Средняя правильность перекрестной проверки (kfold=5): 0.9394
Средняя правильность перекрестной проверки (cv=10): 0.9494
Средняя правильность перекрестной проверки (kfold=10): 0.9516


In [6]:
kfold5 = KFold(n_splits=5, shuffle=True, random_state=42)

cv_scores = cross_val_score(model, X, y, cv=kfold5, scoring='accuracy')
print(f"Средняя правильность перекрестной проверки (kfold=5, shuffle=True): {cv_scores.mean():.4f}")

Средняя правильность перекрестной проверки (kfold=5, shuffle=True): 0.9755


In [7]:
shuffle_split = ShuffleSplit(test_size=.5, train_size=.5, n_splits=10)


cv_scores = cross_val_score(model, X, y, cv=shuffle_split, scoring='accuracy')
print(f"Средняя правильность перекрестной проверки (shuffle_split=10): {cv_scores.mean():.4f}")

Средняя правильность перекрестной проверки (shuffle_split=10): 0.9680


In [12]:
X_train, X_test, y_train, y_test = train_test_split(X, y, random_state=42)

best_score = 0

for gamma in [0.001, 0.01, 0.1, 1, 10, 100]:
    for C in [0.001, 0.01, 0.1, 1, 10, 100]:
        svm = SVC(gamma=gamma, C=C)
        svm.fit(X_train, y_train)
        score = svm.score(X_test, y_test)
            
        if score > best_score:
            best_score = score
            best_parameters = {'C': C, 'gamma': gamma}
            
print(f'Наилучшее значение правильности: {best_score:.4f}')
print(f'Наилучшие значения параметров: {best_parameters}')

Наилучшее значение правильности: 0.9911
Наилучшие значения параметров: {'C': 10, 'gamma': 0.001}


In [13]:
X_train_val, X_test, y_train_val, y_test = train_test_split(X, y, random_state=0)

X_train, X_valid, y_train, y_valid = train_test_split(X, y, random_state=42)

best_score = 0

for gamma in [0.001, 0.01, 0.1, 1, 10, 100]:
    for C in [0.001, 0.01, 0.1, 1, 10, 100]:
        svm = SVC(gamma=gamma, C=C)
        svm.fit(X_train, y_train)
        score = svm.score(X_valid, y_valid)
            
        if score > best_score:
            best_score = score
            best_parameters = {'C': C, 'gamma': gamma}
            
svm = SVC(**best_parameters)
svm.fit(X_train_val, y_train_val)
test_score = svm.score(X_test, y_test)
            
print(f'Лучшее значение правильности на проверочном наборе: {best_score:.4f}')
print(f'Наилучшие значения параметров: {best_parameters}')
print(f'Правильность на тестовом наборе с наилучшими параметрами: {test_score:.4f}')

Лучшее значение правильности на проверочном наборе: 0.9911
Наилучшие значения параметров: {'C': 10, 'gamma': 0.001}
Правильность на тестовом наборе с наилучшими параметрами: 0.9933


In [18]:
param_grid = {
    'C': [0.001, 0.01, 0.1, 1, 10, 100],
    'gamma': [0.001, 0.01, 0.1, 1, 10, 100],
}
X_train, X_test, y_train, y_test = train_test_split(X, y, random_state=42)

grid_search = GridSearchCV(SVC(), param_grid, cv=5)
grid_search.fit(X_train, y_train)

score = grid_search.score(X_test, y_test)
print(f'Правильность на тестовом наборе: {score:.4f}')

print(f'Наилучшие значения параметров: {grid_search.best_params_}')
print(f'Наилучшее значение кросс-валидационной правильности: {grid_search.best_score_:.4f}')

print(f'Наилучшая модель:\n{grid_search.best_estimator_}')

Правильность на тестовом наборе: 0.9911
Наилучшие значения параметров: {'C': 10, 'gamma': 0.001}
Наилучшее значение кросс-валидационной правильности: 0.9889
Наилучшая модель:
SVC(C=10, gamma=0.001)


In [ ]:
target_names = [str(i) for i in range(len(dataset.target_names))]

print("\nОтчет классификации:")
print(classification_report(y_test, y_pred, target_names=target_names))

conf_matrix = confusion_matrix(y_test, y_pred)
print("\nМатрица ошибок:")
print(conf_matrix)

roc_auc = roc_auc_score(y_test, best_model.predict_proba(X_test), multi_class='ovr')
print(f"\nROC-AUC: {roc_auc:.4f}")

accuracy = accuracy_score(y_test, y_pred)
print(f"Точность: {accuracy:.4f}")


Отчет классификации:
              precision    recall  f1-score   support

           0       1.00      0.98      0.99        43
           1       0.95      1.00      0.97        37
           2       1.00      1.00      1.00        38
           3       1.00      0.93      0.97        46
           4       0.98      1.00      0.99        55
           5       0.95      0.98      0.97        59
           6       0.98      0.98      0.98        45
           7       0.98      0.98      0.98        41
           8       0.95      0.95      0.95        38
           9       0.98      0.96      0.97        48

    accuracy                           0.98       450
   macro avg       0.98      0.98      0.98       450
weighted avg       0.98      0.98      0.98       450


Матрица ошибок:
[[42  0  0  0  1  0  0  0  0  0]
 [ 0 37  0  0  0  0  0  0  0  0]
 [ 0  0 38  0  0  0  0  0  0  0]
 [ 0  0  0 43  0  1  0  0  2  0]
 [ 0  0  0  0 55  0  0  0  0  0]
 [ 0  0  0  0  0 58  1  0  0  0]
 [ 0

In [2]:
import pandas as pd
import random

cities = ['Karakol', 'Bishkek', 'Almaty', 'Tashkent', 'Moscow', 'Berlin']
vacation_types = ['Shopping', 'Beach holiday', 'Skiing', 'Cultural trip',
                  'Adventure trip', 'Cruise', 'Road trip']
transport_types = ['auto', 'plane', 'train', 'bus', 'boat', 'yacht', 
                   'bike', 'motorcycle', 'helicopter']
target_cities = ['London', 'Moscow', 'Paris', 'Berlin', 'Amsterdam', 
                 'Madrid', 'Barcelona', 'Rome', 'Dubai', 'Istanbul']

data = {
    'salary': [random.randint(30000, 200000) for _ in range(1000)],
    'city': [random.choice(cities) for _ in range(1000)],
    'age': [random.randint(25, 65) for _ in range(1000)],
    'vacation_prefer': [random.choice(vacation_types) for _ in range(1000)],
    'transport_prefer': [random.choice(transport_types) for _ in range(1000)],
    'target': [random.choice(target_cities) for _ in range(1000)]
}

df = pd.DataFrame(data)
print(df.head()) 

   salary     city  age vacation_prefer transport_prefer  target
0  108721   Moscow   25       Road trip            train   Dubai
1  177552   Berlin   50       Road trip             auto  Moscow
2  113747   Almaty   29   Cultural trip            train   Dubai
3   32572  Karakol   34          Skiing             bike    Rome
4   31124  Bishkek   37   Beach holiday            train   Paris


In [3]:
df_encoded = pd.get_dummies(df, columns=['city', 'vacation_prefer', 'transport_prefer', 'target'])
print(df_encoded.head())  

   salary  age  city_Almaty  city_Berlin  city_Bishkek  city_Karakol  \
0  108721   25        False        False         False         False   
1  177552   50        False         True         False         False   
2  113747   29         True        False         False         False   
3   32572   34        False        False         False          True   
4   31124   37        False        False          True         False   

   city_Moscow  city_Tashkent  vacation_prefer_Adventure trip  \
0         True          False                           False   
1        False          False                           False   
2        False          False                           False   
3        False          False                           False   
4        False          False                           False   

   vacation_prefer_Beach holiday  ...  target_Amsterdam  target_Barcelona  \
0                          False  ...             False             False   
1                     

In [18]:
X = df_encoded.drop(columns=[f'target_{city}' for city in target_cities])
y = df_encoded[[f'target_{city}' for city in target_cities]].idxmax(axis=1)

X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42, stratify=y)

In [25]:
model = RandomForestClassifier(random_state=42)
model.fit(X_train, y_train)

y_pred = model.predict(X_test)
print("Точность:", accuracy_score(y_test, y_pred))
print("\nОтчет классификации:")
print(classification_report(y_test, y_pred))


Точность: 0.14

Отчет классификации:
                  precision    recall  f1-score   support

target_Amsterdam       0.00      0.00      0.00        19
target_Barcelona       0.06      0.05      0.05        19
   target_Berlin       0.10      0.13      0.12        23
    target_Dubai       0.09      0.06      0.07        17
 target_Istanbul       0.18      0.22      0.20        18
   target_London       0.18      0.19      0.19        21
   target_Madrid       0.13      0.20      0.16        20
   target_Moscow       0.07      0.05      0.06        22
    target_Paris       0.24      0.24      0.24        21
     target_Rome       0.23      0.25      0.24        20

        accuracy                           0.14       200
       macro avg       0.13      0.14      0.13       200
    weighted avg       0.13      0.14      0.13       200



In [20]:
random_sample = {
    'salary': [random.randint(30000, 100000)],
    'age': [random.randint(30, 65)],
    'city_Bishkek': [1], 'city_Almaty': [0], 'city_Tashkent': [0], 'city_Astana': [0], 'city_Osh': [0],
    'vacation_prefer_Shopping': [0], 'vacation_prefer_Beach holiday': [1], 'vacation_prefer_Skiing': [0], 'vacation_prefer_Cultural trip': [0],
    'transport_prefer_auto': [1], 'transport_prefer_plane': [0], 'transport_prefer_train': [0], 'transport_prefer_bus': [0]
}

random_sample_df = pd.DataFrame(random_sample)

missing_columns = set(X_train.columns) - set(random_sample_df.columns)
for col in missing_columns:
    random_sample_df[col] = 0

random_sample_df = random_sample_df[X_train.columns]

prediction = model.predict(random_sample_df)
print("Предсказанный город отпуска:", prediction[0])


Предсказанный город отпуска: target_Moscow


In [24]:
param_grid = {
    'n_estimators': [50, 100, 200, 300],  
    'max_depth': [10, 20, 30, None],    
    'min_samples_split': [2, 5, 10, 20], 
    'min_samples_leaf': [1, 2, 4, 8],   
    'bootstrap': [True, False],         
    'class_weight': [None, 'balanced'], 
    'criterion': ['gini', 'entropy'] 
}

grid_search = GridSearchCV(
    estimator=model,
    param_grid=param_grid,
    cv=5,  
    scoring='accuracy',
    n_jobs=-1,  
    verbose=2   
)
grid_search.fit(X_train, y_train)

print("Лучшие параметры:", grid_search.best_params_)
best_model = grid_search.best_estimator_

y_pred_optimized = best_model.predict(X_test)
print("Оптимизированная точность (acc):", accuracy_score(y_test, y_pred_optimized))
print("\nОптимизированный отчет о классификации (CR):")
print(classification_report(y_test, y_pred_optimized))


Fitting 5 folds for each of 2048 candidates, totalling 10240 fits
Лучшие параметры: {'bootstrap': True, 'class_weight': None, 'criterion': 'gini', 'max_depth': 20, 'min_samples_leaf': 2, 'min_samples_split': 2, 'n_estimators': 50}
Оптимизированная точность (acc): 0.12

Оптимизированный отчет о классификации (CR):
                  precision    recall  f1-score   support

target_Amsterdam       0.08      0.05      0.06        19
target_Barcelona       0.11      0.11      0.11        19
   target_Berlin       0.06      0.09      0.07        23
    target_Dubai       0.00      0.00      0.00        17
 target_Istanbul       0.21      0.22      0.22        18
   target_London       0.05      0.05      0.05        21
   target_Madrid       0.09      0.10      0.10        20
   target_Moscow       0.19      0.14      0.16        22
    target_Paris       0.20      0.19      0.20        21
     target_Rome       0.26      0.25      0.26        20

        accuracy                           0.

In [29]:
cv_scores = cross_val_score(model, X, y, cv=5, scoring='accuracy')
print("Средняя точность перекрестной проверки:", cv_scores.mean())

Средняя точность перекрестной проверки: 0.126


In [30]:
kfold5 = KFold(n_splits=5, shuffle=True, random_state=42)

cv_scores = cross_val_score(model, X, y, cv=kfold5, scoring='accuracy')
print(f"Средняя правильность перекрестной проверки (kfold=5, shuffle=True): {cv_scores.mean():.4f}")

Средняя правильность перекрестной проверки (kfold=5, shuffle=True): 0.1270


## **Лабораторная работа №6**
### **Вывод:**

В данной лабораторной работе использовался датасет **digits** и в качестве модели **RandomForestClassifier**.

В первой части необходимо было обучить модель, а затем улучшить ее точность с помощью методов **перекрестной проверки**, **решетчатого поиска** и **метрик модели**.

---

#### Перекрестная проверка:

| Тип перекрестной проверки | Точность | Параметр   		  |  
| ------------------------- | -------- | -------------------- | 
| Обычная 			  		| 0.9394   | **cv=5**     		  | 
| Обычная 			  		| 0.9494   | **cv=10**     		  | 
| Стратифицированная  		| 0.9394   | **kfold=5**  		  | 
| Стратифицированная  		| 0.9516   | **kfold=10**  		  | 
| Страт. с перестановками   | 0.9755   | **kfold=5** 		  | 
| С перестановками и разб.  | 0.9680   | **shuffle_split=10** | 

#### Решетчатый поиск:
 
| Тип 					    | Точность | Лучшие параметры      |
| ------------------------- | -------- | --------------------- |
| Без валидационного набора | 0.9911   | **C=10; gamma=0.001** |
| С валидационным набором   | 0.9933   | **C=10; gamma=0.001** | 

#### Метрики:

Отчет классификации:

              precision    recall  f1-score   support

           0       1.00      0.98      0.99        43
           1       0.95      1.00      0.97        37
           2       1.00      1.00      1.00        38
           3       1.00      0.93      0.97        46
           4       0.98      1.00      0.99        55
           5       0.95      0.98      0.97        59
           6       0.98      0.98      0.98        45
           7       0.98      0.98      0.98        41
           8       0.95      0.95      0.95        38
           9       0.98      0.96      0.97        48           
      accuracy                         0.98       450 
      macro avg    0.98      0.98      0.98       450
      weighted avg 0.98      0.98      0.98       450

**ROC-AUC = 0.9996** | **Accuracy = 0.9756**

В целом модель показывает хорошие результаты на датасете с цифрами.

---

Во второй части необходимо было сгенерировать свой датасет со случайными несбалансированными данными **(salary, city, age, vacation_prefer, transport_prefer, target)** и натренировать модель уже на них. 

Модель должна определить город, в который скорее всего отправиться человек для проведения отдыха.

---

#### Часть данных:

      salary     city  age vacation_prefer    transport_prefer  target
      0  108721   Moscow   25       Road trip            train   Dubai
      1  177552   Berlin   50       Road trip             auto  Moscow
      2  113747   Almaty   29   Cultural trip            train   Dubai
      3   32572  Karakol   34          Skiing             bike    Rome
      4   31124  Bishkek   37   Beach holiday            train   Paris
    
#### Результаты:

Отчет классификации:
                        
                        precision    recall  f1-score   support
    target_Amsterdam      0.00      0.00      0.00        19
    target_Barcelona      0.06      0.05      0.05        19
    target_Berlin         0.10      0.13      0.12        23
    target_Dubai          0.09      0.06      0.07        17
    target_Istanbul       0.18      0.22      0.20        18
    target_London         0.18      0.19      0.19        21
    target_Madrid         0.13      0.20      0.16        20
    target_Moscow         0.07      0.05      0.06        22
    target_Paris          0.24      0.24      0.24        21
    target_Rome           0.23      0.25      0.24        20
    
        accuracy                              0.14       200
       macro avg          0.13      0.14      0.13       200
    weighted avg          0.13      0.14      0.13       200

**Accuracy = 0.14**

На несбалансированных данных модель показывает гораздо более худшие результаты нежели на уже подготовленных. 

После попытки улучшения модели методом решетчатого поиска ее точность наоборот понизилась **(accuracy = 0.12)**.

Метод перекрестной проверки выдает лишь немногим большую точность 0.126.

Несбалансированность данных сильно ухудшает предсказательную способность модели и возможность ее улучшения.